In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.optimize as opt
from scipy.io import loadmat
from sklearn.svm import SVC
from sklearn import decomposition
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
#Coge el archivo abalone.txt y sustituye F M e I
#por valores numéricos que pueda leer luego
#Almacena la sustitución en abaloneProcessed.txt
def preprocess():
    f = open("abalone.txt", 'r')
    o = open("abaloneProcessed.txt", 'w')
    c = f.read(1)
    while True:
        if not c:
            break
        if c == 'M':
            o.write('-1')
        elif c == 'F':
            o.write('1')
        elif c == 'I':
            o.write('0')
        else:
            o.write(c)
        c = f.read(1)
    f.close()
    o.close()
    
preprocess()

In [ ]:
#Precondición 
#0 < Numtraining < 3800; 0 < components < 9; 1 < groupsize < 30
#Carga los datos aleatoriamente
def loadDataShuffled(decompose=False, components=2, shrinky=False, 
                     groupSize=2, numTraining = 3000):
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    
    x = data[:,:-1]
    y = data[:,-1]
    
    #Si se quiere reducir los parámetros con PCA
    if decompose == True:
        pca = decomposition.PCA(n_components = components)
        pca.fit(x)
        x = pca.transform(x)
    
    #Si se quiere agrupar la y en grupos
    if shrinky == True:
        y = y // groupSize
    
    join = np.hstack((x, np.c_[y]))
    np.random.shuffle(join)
    
    #Se vuelven a separar tras barajearse
    xShuffled = join[:,:-1]
    yShuffled = join[:,-1]
    
    #numTraining decide cuantos ejemplos hay para entrenamiento,
    #y cuantos hay para validación (3800 - numTraining)
    xTraining = xShuffled[:numTraining]
    yTraining = yShuffled[:numTraining]
    xVal = xShuffled[numTraining:3800]
    yVal = yShuffled[numTraining:3800]
    xTest = xShuffled[3800:]
    yTest = yShuffled[3800:]
    
    return xTraining, yTraining, xVal, yVal, xTest, yTest

In [ ]:
#Carga para entrenar las muestras distintas a lo normal,
#y para validar las normales.
def loadDataTrainOdd():
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    
    x = data[:,:-1]
    y = data[:,-1]
    
    xTraining = np.empty((0, x.shape[1]))
    yTraining = np.array([])
    xVal = np.empty((0, x.shape[1]))
    yVal = np.array([])
    xTest = np.empty((0, x.shape[1]))
    yTest = np.array([])
    
    for i in range(x.shape[0]):
        #Valores extraidos a partir de la desviación 
        #típcia y la media de y
        if y[i] < 5 or y[i] > 15:
            xTraining = np.vstack((xTraining, x[i]))
            yTraining = np.append(yTraining, y[i])
        elif random.randint(0,1) == 1:
            xVal = np.vstack((xVal, x[i]))
            yVal = np.append(yVal, y[i])
        else: 
            xTest = np.vstack((xTest, x[i]))
            yTest = np.append(yTest, y[i])
    return xTraining, yTraining, xVal, yVal, xTest, yTest

In [ ]:
#Carga para entrenar las muestras normales,
#y para validar las raras.
def loadDataTrainNormal():
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    
    x = data[:,:-1]
    y = data[:,-1]
    
    print(np.mean(y), np.std(y))
    
    xTraining = np.empty((0, x.shape[1]))
    yTraining = np.array([])
    xVal = np.empty((0, x.shape[1]))
    yVal = np.array([])
    xTest = np.empty((0, x.shape[1]))
    yTest = np.array([])
    
    for i in range(x.shape[0]):
        
        if y[i] >= 5 and y[i] <= 15:
            xTraining = np.vstack((xTraining, x[i]))
            yTraining = np.append(yTraining, y[i])
        elif random.randint(0,1) == 1:
            xVal = np.vstack((xVal, x[i]))
            yVal = np.append(yVal, y[i])
        else: 
            xTest = np.vstack((xTest, x[i]))
            yTest = np.append(yTest, y[i])
    return xTraining, yTraining, xVal, yVal, xTest, yTest

In [ ]:
#Muestra en un gráfico la distribución del número de ejemplos 
#respecto al número de anillos.
def showDistribution():
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    y = data[:,-1]
    
    distinctY = np.unique(y)
    count = np.zeros_like(distinctY)
    k = 0
    for i in distinctY:
        count[k] = np.sum(y == i)
        k = k + 1
    print(count)
    plt.figure()
    plt.title("Distribution of examples")
    plt.ylabel("Number of examples")
    plt.xlabel("Number of rings")
    plt.bar(distinctY, count, color="cornflowerblue")
    plt.savefig("graph/distribution")

showDistribution()

In [ ]:
#Muestra la correlación de un atributo con
#el número de anillos
def showCorrelation(attribute):
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    y = data[:,-1]
    x = data[:,:-1]
    
    plt.figure()
    plt.plot(x[:,attribute], y, "x", c="tomato")
    plt.ylabel("Number of Rings")
    plt.title("Correlation with shell weight")
    plt.xlabel("Shell Weight (after being dried) (g)")
    plt.savefig("graph/ShellWeight")
    
showCorrelation(7)

In [ ]:
#Asume que esta hecho con -1 Male, 0 Infant, 1 Female
#Muestra cuantos ejemplos hay para cada sexo
def showExamplesEachSex():
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    y = data[:,-1]
    x = data[:,:-1]
    
    sex = np.array([-1, 0, 1])
    number = np.zeros_like(sex)
    number[0] = np.sum(x[:,0] == -1)
    number[1] = np.sum(x[:,0] == 0)
    number[2] = np.sum(x[:,0] == 1)
    plt.figure()
    plt.title("Distribution for sex")
    plt.ylabel("Number of examples")
    plt.xlabel("Sex (-1 = Male; 0 = Infant; 1 = Female)")
    plt.bar(sex, number, color="cornflowerblue")
    plt.savefig("graph/distributionSex")

showExamplesEachSex()

# SVM

In [ ]:
#Porcentaje de correctos para la SVM, teniendo en cuenta
#que consideramos correctos los que predigan +- rango
#del valor correcto
def correctPercentage(svm, x, y, rango):
    array = svm.predict(x)
    cmp = (np.abs(array - y) <= rango) 
    return np.sum(cmp)/len(y)*100

In [ ]:
#Cantidad de error, hace que los que se alejen demasiado 
#lo aumenten exponencialmente más.
def errorAmmountSVM(svm, x, y):
    array = svm.predict(x)
    return np.sum(np.abs(array - y) ** 2)

In [ ]:
#Pone ejemplos, predicciones hechas por la svm y 
#el valor real.
def printExamplesSVM(svm, x, y):
    array = svm.predict(x)
    k = 0
    for i in array:
        print("Valores de x: ", x[k])
        print("El valor predicho es: %i" % i)
        print("El valor real es: %.0f" % y[k])
        k = k+1

In [ ]:
#Implementación de una SVM, dibujando variós gráficos, incluida
#uno en 3d sobre la efectividad de C y sigma 
def SVM(rango): 
    xTraining,yTraining,xVal,yVal,xTest,yTest = loadDataShuffled(numTraining=3000)
    bestvalue = -1
    nums = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30])
    k = 0
    bestValuesMatrix = np.array([])
    bestValuesMatrixT = np.array([])

    #Prueba para todos los sigmas con todos los C
    for sigma in nums:
        plt.figure()
        arrayPer = np.array([])
        arrayPerTraining = np.array([])
        bestValuesArray = np.array([])
        
        plt.xlabel("C")
        plt.ylabel("Correct percentage (error margin = %i)" % rango)
        plt.title("Sigma = %.2f" % sigma)
        for c in nums:
            svm = SVC(kernel='rbf', C=c ,gamma=1 / (2 * sigma**2))
            svm.fit(xTraining,yTraining)
            per = correctPercentage(svm,xVal,yVal, rango)
            perEnt = correctPercentage(svm,xTraining,yTraining, rango)
            arrayPer = np.append(arrayPer, per)
            arrayPerTraining = np.append(arrayPerTraining, perEnt)
            if per >= bestvalue:
                bestc = c
                bestsigma = sigma
                bestvalue = per
                bestsvm = svm
            print("%.2f para C = %.2f y sigma = %.2f" % (per, c, sigma))
            print("#%.2f para C = %.2f y sigma = %.2f (entrenamiento)" % (perEnt, 
                                                                          c, sigma))
            print("errorAmmount", errorAmmountSVM(svm, xVal, yVal))
            
        bestValuesMatrix = np.append(bestValuesMatrix, arrayPer)
        bestValuesMatrixT = np.append(bestValuesMatrixT, arrayPerTraining)

        #Dibuja la figura en 2D, comparando cada sigma con cada C
        plt.plot(nums, arrayPer, "-", color="orange" ,label="Validation")
        plt.plot(nums, arrayPerTraining, "-", color="blue" ,label="Training")
        plt.legend()
        plt.savefig("svm/sigma%i.png" % k)
        k = k + 1

    #Dibuja en 3D:
    #Los ejemplos de Validación
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    sigma2, c2 = np.meshgrid(nums, nums)
    bestValuesMatrix = np.reshape(bestValuesMatrix, c2.shape)
    ax.plot_surface(c2, sigma2,
                    bestValuesMatrix, cmap=cm.coolwarm, 
                          linewidth=0, antialiased=False)
    
    ax.set_xlabel("Sigma")
    ax.set_ylabel("C")
    ax.set_zlabel("Correct percentage")
    ax.set_title("Effects on validation examples")
    
    #Los ejemplos de entrenamiento
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    bestValuesMatrixT = np.reshape(bestValuesMatrixT, c2.shape)
    ax.plot_surface(c2, sigma2,
                    bestValuesMatrixT, cmap=cm.coolwarm, 
                          linewidth=0, antialiased=False)
    
    ax.set_xlabel("Sigma")
    ax.set_ylabel("C")
    ax.set_zlabel("Correct percentage")
    ax.set_title("Effects on training examples")
    
    print("Mejor porcentaje de acierto sobre los ejemplos de validación: %.2f" % bestvalue)
    
    svm = SVC(kernel='rbf', C=30 ,gamma=1 / (2 * 0.01**2))
    svm.fit(xTraining,yTraining)

    
    
SVM(0)

In [ ]:
#Otra implementación igual que la anterior, sin generar gráficos
#Incluye opción para pasar los ejemplos por parámetro(data),
#Probar con diferentes números de componentes(componentes y Trycomponentes debe
#ser True), y para probar con distintos tamaños de grupo con groupsize
def SVM2(rango, data=None, TryComponents=False,componentes = 8, groupsize = 1): 
    if data is None:
        xTraining,yTraining,xVal,yVal,xTest,yTest = loadDataShuffled(TryComponents,
                                                                     componentes, 
                                                                     TryComponents, 
                                                                     groupsize)
    else: 
        xTraining,yTraining,xVal,yVal,xTest,yTest = data
        
    bestvalue = -1
    nums = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30])
    
    for c in nums:
        for sigma in nums:
            svm = SVC(kernel='rbf', C=c ,gamma=1 / (2 * sigma**2), max_iter=-1)
            svm.fit(xTraining,yTraining)
            per = correctPercentage(svm,xVal,yVal, rango)  
            if per >= bestvalue:
                bestc = c
                bestsigma = sigma
                bestvalue = per
                bestsvm = svm
    
    #Imprime 10 ejemplos
    #printExamplesSVM(bestsvm, xTest[0:10], yTest[0:10])
    
    #print("Mejor porcentaje de acierto sobre los ejemplos de validación: %.2f" % bestvalue)
    
    return bestvalue

In [ ]:
#Busca el mejor resultado para el kernel linear y gaussiano y lo devuelve
def linearVSGaussian(rango, numT):
    xTraining, yTraining, xVal, yVal, xTest, yTest = loadDataShuffled(numTraining=numT)
    bestvalue = -1
    nums = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30])
    bestvalueLinear = -1
    for c in nums:
        for sigma in nums:
            svm = SVC(kernel='rbf', C=c ,gamma=1 / (2 * sigma**2), max_iter=-1)
            svm.fit(xTraining,yTraining)
            per = correctPercentage(svm,xVal,yVal, rango)  
            if per >= bestvalue:
                bestc = c
                bestsigma = sigma
                bestvalue = per
        svmLinear = SVC(kernel='linear', C=c)
        svmLinear.fit(xTraining, yTraining)
        perLinear = correctPercentage(svmLinear, xVal, yVal, rango)
        if perLinear >= bestvalueLinear:
            bestcLinear = c
            bestvalueLinear = perLinear
        
    
    #print("Mejor porcentaje de acierto sobre los ejemplos de validación: %.2f" % bestvalue)
    #print("Mejor porcentaje de acierto sobre los ejemplos de validación (Linear):
    #%.2f" % bestvalueLinear)
    
    return bestvalue, bestvalueLinear

#Ejecuta varias veces linearVSGaussian() para hacer gráficos, comparando
#Los resultados de utilizar cada kernel
def linearVSGaussianGraphic():
    arrayGaussian = np.array([])
    arrayLinear = np.array([])
    arrayI = np.array([])
    for i in range(10):
        resGaussian, resLinear = linearVSGaussian(2, 100)
        arrayGaussian = np.append(arrayGaussian, resGaussian)
        arrayLinear = np.append(arrayLinear, resLinear)
        arrayI = np.append(arrayI, i * 2)
    
    plt.figure()
    plt.title("Linear vs Gaussian kernel")
    plt.bar(arrayI, arrayGaussian, color = "sandybrown", label="Gaussian")
    plt.bar(arrayI + 1, arrayLinear, color = "seagreen", label="Linear")
    plt.ylabel("Correct percentage; range = 2")
    plt.legend()
    plt.xticks()
    plt.ylim((70,80))
    plt.savefig("svm/LinearGaussian2")
    
linearVSGaussianGraphic()   

In [ ]:
#Compara la efectividad para cada numero de componentes
#dibujando un gráfico sobre ello
def aciertosVSnComponentes():
    plt.figure()
    bestValues = np.array([])
    component = np.array([])
    for i in range(1, 9):
        bestValues = np.append(bestValues, SVM2(0, TryComponents=True, componentes = i))
        component = np.append(component, i)
    plt.title("Correct percentage vs number of components")
    plt.ylabel("Correct Percentage")
    plt.xlabel("Number of components")
    plt.ylim((np.min(bestValues) - 2, np.max(bestValues) + 2))
    plt.bar(component, bestValues, color="cornflowerblue")
    plt.savefig("svm/AciertosVSComponentes3.png")
    
aciertosVSnComponentes()

In [ ]:
SVM2(3.)

In [ ]:
SVM(2.)

# Neural network

In [ ]:
def sigmoide(x):
    return 1./(1. + np.exp(-1. * x))

In [ ]:
#Genera una theta inicial con valores aleatorios
def pesosAleatorios(Lin, Lout):
    ini = 0.12 
    theta = np.random.random((Lout,Lin+1))*(2*ini)-ini
    return theta

In [ ]:
def h0(x,theta1,theta2):
    z2 = np.matmul(x, theta1.T)
    a2 = sigmoide(z2)
    a2 = np.hstack((np.ones((a2.shape[0],1)), a2))
    
    #Capa de salida
    z3 = np.matmul(theta2, a2.T)
    a3 = sigmoide(z3)
    
    return a3, a2 

In [ ]:
#Solo funciona para 1 capa oculta
def backdrop(params_rn, num_entradas, num_ocultas, num_etiquetas, x, y, mini, reg = 0 ):
    #Desempaqueta los parámetros
    theta1 = np.reshape(params_rn[: num_ocultas * (num_entradas + 1) ],
                        (num_ocultas , (num_entradas + 1)))
    theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1) :],
                        (num_etiquetas , (num_ocultas + 1)))
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))

    #Genera la matriz de etiquetas.
    y2 = np.zeros((len(y), num_etiquetas))
    #Pone los 1 en la matriz de 0
    for i in range(len(y)):
        y2[i,int(y[i]) - mini] = 1
    
    #FORWARD PROPAGATION-------------------------------------------------
    a3, a2 = h0(xOnes, theta1, theta2)
    a = - y2 * np.log(a3).T
    b = (1 - y2) * np.log(1 - a3).T
    c = np.sum(1. / (xOnes.shape[0]) * (a - b))
    coste = c + (reg/(2 * xOnes.shape[0]))*(np.sum(theta1**2) + np.sum(theta2**2))

    #BACKPROPAGATION------------------------------------------------------
    delta3 = a3 - y2.T
    delta2 = np.matmul(theta2.T, delta3) * (a2.T * (1. - a2.T))
    #Copia theta para evitar dañar la original para vectorizar que
    #la primera fila no se modifica al regularizar
    theta1c = np.copy(theta1)
    theta2c = np.copy(theta2)
    theta2c[:,0] = theta2c[:,0] * 0  
    theta1c[:,0] = theta1c[:,0] * 0
    
    #Calcula las theta resultantes, incluyendo la regularización
    triangulo2 = (np.matmul(delta3, a2) / xOnes.shape[0]) 
    triangulo2 = triangulo2 + (reg/xOnes.shape[0]) * theta2c
    triangulo1 = (np.matmul(delta2[1:], xOnes) / xOnes.shape[0]) 
    triangulo1 = triangulo1 + (reg/xOnes.shape[0]) * theta1c
    
    return coste, np.concatenate((triangulo1.ravel(),triangulo2.ravel()))

In [ ]:
def coste(xOnes, y2, theta1, theta2, reg):
    a3, a2 = h0(xOnes, theta1, theta2)
    a = - y2 * np.log(a3).T
    b = (1 - y2) * np.log(1 - a3).T
    c = np.sum(1. / (xOnes.shape[0]) * (a - b))
    coste = c + (reg/(2 * xOnes.shape[0]))*(np.sum(theta1**2) + np.sum(theta2**2))
    return coste

In [ ]:
#printExamplesSVM pero para Redes neuronales
def checkNeural(theta_opt1, theta_opt2, x, y, maxi, mini):
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    for i in range(xOnes.shape[0]):
        salida = h0(xOnes[i,np.newaxis], theta_opt1, theta_opt2)
        salida = salida[0].ravel()
        print("Valores de x: ", xOnes[i])
        print("El valor predicho es: %i" % (salida.argmax() + mini % maxi))
        print("El valor real es: %i" % int(y[i]))


In [ ]:
#correctPercentage pero para redes neuronales
def correctPercentageNN(theta_opt1, theta_opt2, x, y, maxi, mini, rango):
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    array = h0(xOnes, theta_opt1, theta_opt2)[0]

    result = (np.argmax(array, axis=0) + mini) % maxi
    
    cmp = np.abs(y - (result)) <= rango 
    return np.sum(cmp)/len(y)*100 

In [ ]:
#errorAmmountSVM pero para redes neuronales
def errorAmmountNN(theta_opt1, theta_opt2, x, y, maxi, mini):
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    array = h0(xOnes, theta_opt1, theta_opt2)[0]

    result = (np.argmax(array, axis=0) + mini) % maxi
    return np.sum(np.abs(result - y) ** 2)

In [ ]:
#Entrena una red neuronal para un valor discreto de regularización
def neuralNetwork(reg, num_ocultas, rango, data = None):
    
    if data is None:
        xTraining, yTraining, xVal, yVal, xTest, yTest = loadDataShuffled()
    else:
        xTraining, yTraining, xVal, yVal, xTest, yTest = data
    num_entradas = xTraining.shape[1]
        
    #Utiliza el mínimo y máximo para crear el número de etiquetas
    maxi = int(np.max([np.max(yTraining), np.max(yVal), np.max(yTest)]))
    mini = int(np.min([np.min(yTraining), np.min(yVal), np.min(yTest)]))
    num_etiquetas = maxi - mini + 1
        
    t1 = pesosAleatorios(num_entradas,num_ocultas)
    t2 = pesosAleatorios(num_ocultas, num_etiquetas)
    params_t = np.vstack((np.reshape(t1, (t1.shape[0]*t1.shape[1],1)),
                           np.reshape(t2, (t2.shape[0]*t2.shape[1],1))))
    
    #Entrena la red neuronal
    result = opt.minimize(fun=backdrop, x0=params_t,
                          args=(num_entradas, num_ocultas,
                                num_etiquetas, xTraining, yTraining, mini, reg),
                          method='TNC', jac=True)
    
    #Desempaqueta la salida del entrenamiento
    salida1 = np.reshape(result.x[: num_ocultas * (num_entradas + 1) ], 
                         (num_ocultas , (num_entradas + 1)))
    salida2 = np.reshape(result.x[num_ocultas * (num_entradas + 1) :], 
                         (num_etiquetas , (num_ocultas + 1)))
    
    #Imprime 10 ejemplos de test
    #checkNeural(salida1, salida2, xTest[0:10], yTest[0:10], maxi, mini)
    
    return correctPercentageNN(salida1, salida2, xVal, yVal, maxi, mini,0), result.fun




In [ ]:
#Busca el mejor valor probando con distintas lambdas.
#Puedes decidir si se dibujan gráficas o no con plot
def neuralNetworkDecide(maxiter, num_ocultas, rango, data=None, plot=True):
    
    if data is None:
        xTraining, yTraining, xVal, yVal, xTest, yTest =  loadDataShuffled()
    else:
        xTraining, yTraining, xVal, yVal, xTest, yTest = data
    
    num_entradas = xTraining.shape[1]
    
    #Utiliza el mínimo y máximo para crear el número de etiquetas,
    #Se utiliza el minimo y máximo entre todos los ejemplos, aunque
    #no aparezcan en los de entranamineto
    maxi = int(np.max([np.max(yTraining), np.max(yVal), np.max(yTest)]))
    mini = int(np.min([np.min(yTraining), np.min(yVal), np.min(yTest)]))
    num_etiquetas = maxi - mini + 1
    
    
    xValOnes = np.hstack((np.ones((xVal.shape[0],1)), xVal))

    #Genera la matriz de etiquetas.
    yVal2 = np.zeros((len(yVal), num_etiquetas))
    #Pone los 1 en la matriz de 0
    for i in range(len(yVal)):
        yVal2[i,int(yVal[i]) - mini] = 1
    
    reg = np.array([0.001, 0.003, 0.007, 0.01, 0.03, 0.07, 
                    0.1, 0.3, 0.7, 1, 3, 7, 10, 30])
    
    #Array para almacenar el resultado del coste para cada lambda
    errorTra = np.array([])
    errorVal = np.array([])
    correctPerTra = np.array([])
    correctPerVal = np.array([])
    #Entrena la red neuronal
    t1 = pesosAleatorios(num_entradas,num_ocultas)
    t2 = pesosAleatorios(num_ocultas, num_etiquetas)
    params_t = np.vstack((np.reshape(t1, (t1.shape[0]*t1.shape[1],1)),
                          np.reshape(t2, (t2.shape[0]*t2.shape[1],1))))
    for i in reg:
  
        result = opt.minimize(fun=backdrop, x0=params_t,
                              args=(num_entradas, num_ocultas,
                                    num_etiquetas, xTraining, yTraining, mini, i),
                              method='TNC', jac=True, options={'maxiter': maxiter})
    
    #Desempaqueta la salida del entrenamiento
        salida1 = np.reshape(result.x[: num_ocultas * (num_entradas + 1) ], 
                             (num_ocultas , (num_entradas + 1)))
        salida2 = np.reshape(result.x[num_ocultas * (num_entradas + 1) :], 
                             (num_etiquetas , (num_ocultas + 1)))
        correctPerVal = np.append(correctPerVal,
                                  correctPercentageNN(salida1, salida2,
                                                      xVal, yVal,
                                                      maxi, mini, rango))
        correctPerTra = np.append(correctPerTra,
                                  correctPercentageNN(salida1, salida2,
                                                      xTraining, yTraining,
                                                      maxi, mini, rango))

        #print(correctPercentageNN(salida1, salida2, xVal, yVal, maxi, mini, rango))
        cost = coste(xValOnes, yVal2, salida1, salida2, i)
        errorTra = np.append(errorTra, [result.fun])
        errorVal = np.append(errorVal, [cost])
        #print("%.5f para reg = %.3f" % (cost, i))
    if plot:
        #Gráfico con el error
        plt.figure()
        plt.plot(reg, errorTra, "-", color="blue", label="Training")
        plt.plot(reg, errorVal, "-", color ="orange", label="Validation")
        plt.title("Error for neural network")
        plt.xlabel("Regularization value")
        plt.legend()
        plt.ylabel("Error")
        plt.xscale("log")
        plt.savefig("nn/reg1")

        #Gráfico con el porcentaje de correctos
        plt.figure()
        plt.plot(reg, correctPerTra, "-", color="lightblue", label="Training")
        plt.plot(reg, correctPerVal, "-", color ="tomato", label="Validation")
        plt.title("Correct Percentage for neural network")
        plt.xlabel("Regularization value")
        plt.legend()
        plt.ylabel("Correct Percentage (range=2)")
        plt.xscale("log")
        plt.savefig("nn/reg2")
    #checkNeural(salida1, salida2, xVal[0:20], yVal[0:20], maxi, mini)
    return np.max(correctPerVal)#salida1, salida2, result.fun, maxi, mini

In [ ]:
#Compara la efectividad para cada numero de nodos en la capa oculta.
#Muestra dos graficos con los mismos datos.
def aciertosVSnOcultas():
    bestValues = np.array([])
    minError = np.array([])
    component = np.array([])
    datos = loadDataShuffled()

    for i in range(1,11):
        res = neuralNetwork(0., i, 0, data=datos)
        bestValues = np.append(bestValues, res[0])
        minError = np.append(minError, res[1])
        
        component = np.append(component, i)

    #Gráfica para los porcentajes
    plt.figure()
    plt.title("Number of hidden nodes effects")
    plt.ylabel("Correct Percentage")
    plt.xlabel("Number of hidden nodes")
    plt.ylim((np.min(bestValues) - 0.2, np.max(bestValues) + 0.2))
    plt.bar(component, bestValues, color="cornflowerblue")
    plt.savefig("nn/AciertosVSnOcultas10.png")
    
    #Gráfica para los errores
    plt.figure()
    plt.title("Number of hidden nodes effects")
    plt.ylabel("Error value")
    plt.xlabel("Number of hidden nodes")
    plt.ylim((np.min(minError) - 0.2, np.max(minError) + 0.2))
    plt.bar(component, minError, color="yellowgreen")
    plt.savefig("nn/AciertosVSnOcultas20.png")
    
    
aciertosVSnOcultas()

In [ ]:
res = neuralNetworkDecide(1000, 15, 2)

In [ ]:
neuralNetwork(0.01, 15, 0)

# Regresión logistica multicase

In [ ]:
#Función de coste
def J(theta, x, y, landa):
    theta = np.c_[theta] 
    aux = sigmoide(np.matmul(x, theta))
    a = np.log(aux) * y
    b = np.log(1 - aux) * (1 - y)
    c = - 1 / y.shape[0] * (a + b)
    d = landa / (2 * y.shape[0]) * np.sum(theta**2)
    return np.sum(c) + d

In [ ]:
#Gradiente
def descenso(theta, x, y, landa):
    theta = np.c_[theta]
    a = sigmoide(np.matmul(x,theta)) - y
    b = np.matmul(np.transpose(x),a)
    c = (1/y.shape[0] * b)
    d = landa/y.shape[0] * theta
    d[0,0] = 0 
    return np.ravel(c + d)

In [ ]:
def oneVsAll(x, y, num_etiquetas, reg, mini):
    #Crea theta
    theta = np.ravel(np.zeros((1,x.shape[1])))
    
    #Genera la matriz de etiquetas.
    y2 = np.zeros((len(y), num_etiquetas))
    #Pone los 1 en la matriz de 0
    for i in range(len(y)):
        y2[i,int(y[i]) - mini] = 1
    
        
    #Crea theta auxiliar para ir guardando los 
    #resultados de cada optimizacion
    theta_opt = np.reshape(theta, (len(theta),1))
    
    #Ahora para cada etiqueta hacemos el descenso
    for i in range(num_etiquetas):
        result = opt.fmin_tnc(func=J , x0=theta, fprime=descenso, 
                              args=(x, np.c_[y2[:,i]], reg)) 
        theta_opt = np.hstack((theta_opt, np.reshape(result[0],
                                                     (len(result[0]),1))))
    
    #Se devuelve todo menos la primera fila, que se usaba como auxiliar 
    #para ir creando la matriz
    return theta_opt[:,1:] 

In [ ]:
def train(x,y,reg,num_etiquetas, mini):
    #Vector de X con unos para operar con theta
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    
    #Devuelve el vector optimizado
    return oneVsAll(xOnes,y, num_etiquetas,reg, mini)

In [ ]:
#printExamplesSVM o checkNeural para regresión logística
def doExamples(x, y, theta, mini, maxi):
    
    #Vector de x con unos para operar matricialmente con theta
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    
    for i in range(xOnes.shape[0]):
        salida = sigmoide(np.matmul(xOnes[i], theta))
        print("Valores de x: ", xOnes[i])
        print("El valor predicho es: %i" % ((salida.argmax() + mini) % maxi))
        print("El valor real es: %i" % int(y[i]))


In [ ]:
#correctPercentage o correctPercentageNN para regresión logística
def correctPercentageRL(theta, x, y, maxi, mini, rango):
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))

    array = sigmoide(np.matmul(xOnes, theta))
    result = (np.argmax(array, axis=1) + mini) % maxi
    
    cmp = np.abs(y - (result)) <= rango 
    return np.sum(cmp)/len(y)*100

In [ ]:
#Entrena para un valor discreto de reg
def regresionLogistica(reg, rango, data=None):
    
    if data is None:
        xtrain, ytrain, xval, yval, xtest, ytest = loadDataShuffled()
    else:
        xtrain, ytrain, xval, yval, xtest, ytest = data
        
    #Entrena con los ejemplos dados y lambda
    maxi = int(np.max([np.max(ytrain), np.max(yval), np.max(ytest)]))
    mini = int(np.min([np.min(ytrain), np.min(yval), np.min(ytest)]))
    num_etiquetas = maxi - mini + 1
    theta = train(xtrain, ytrain, reg, num_etiquetas, mini)
    
    trainPer = correctPercentageRL(theta, xtrain, ytrain, maxi, mini, rango)
    return correctPercentageRL(theta, xval, yval, maxi, mini, rango), trainPer, theta



In [ ]:
#Busca el mejor resultado probando con varios valores de lambda
def chooseLanda(numTrain, rango, data=None, plot=True):
    reg = np.array([0.001, 0.003, 0.007, 0.01, 0.03, 0.07, 
                    0.1, 0.3, 0.7, 1, 3, 7, 10, 30])
    if data is None:
        data = loadDataShuffled(numTraining=numTrain)
        
    xtrain, ytrain, xval, yval, xtest, ytest = data
    maxi = int(np.max([np.max(ytrain), np.max(yval), np.max(ytest)]))
    mini = int(np.min([np.min(ytrain), np.min(yval), np.min(ytest)]))
    
    values = np.array([])
    valuesTrain = np.array([])
    bestPer = -1
    for i in reg:
        RL = regresionLogistica(i,rango,data)
        values = np.append(values, RL[0])
        valuesTrain = np.append(valuesTrain, RL[1])
        if RL[0] > bestPer:
            bestPer = RL[0]
            bestPerT = RL[1]
            bestTheta = RL[2]
            bestLan = i
    
    if plot:
        plt.figure()
        plt.xscale("log")
        plt.plot(reg, values, "-", c="khaki", label="Validation")
        plt.plot(reg, valuesTrain, "-", c="turquoise", label="Training")
        plt.legend()
        plt.xlabel("Regularization value")
        plt.ylabel("Correct percentage (range=%i)" % rango)
        plt.title("Number of examples: %i" % numTrain)
        print("Best lambda = %.3f" % bestLan)
        plt.savefig("rl/correctPercentage%i" % numTrain)
    
    #doExamples(xtest[0:10],ytest[0:10],bestTheta, mini, maxi)
    
    return bestPer, bestPerT



In [ ]:
#Preprocesa los ejemplos para hacer polinomios de grado numpoly
#y hace la regresión discreta con los parámetros dados
def regresionLogisticaPoly(reg, numpoly,data,rango):
    xtrain, ytrain, xval, yval, xtest, ytest = data
    poly = PolynomialFeatures(numpoly) #Función polinomial
    xtrain = poly.fit_transform(xtrain)
    xval = poly.fit_transform(xval)
    xtest = poly.fit_transform(xtest)
    datos = xtrain, ytrain, xval, yval, xtest, ytest
    RL = regresionLogistica(reg,rango,datos)
    return RL[0], RL[1]

In [ ]:
#Busca el mejor tamaño para el polinomio; dibuja la gráfica
def bestNumPoly(rango):
    percentages = np.array([])
    percentagesT = np.array([])
    k = np.array([])

    data = loadDataShuffled()
    
    #Solo 4, por el gran coste computacional.
    for i in range(1,5):
        bestValues = regresionLogisticaPoly(0.01,i,data,rango)
        percentages = np.append(percentages, bestValues[0])
        percentagesT = np.append(percentagesT, bestValues[1])
        k = np.append(k, i)
        
    plt.figure()
    plt.xlabel("Polinomy degree")
    plt.ylabel("Correct Percentage (range=%i)" % rango)
    plt.plot(k, percentages, "-", c="darkorchid", label="Validation")
    plt.plot(k, percentagesT, "-", c="greenyellow", label="Training")
    plt.legend()
    plt.savefig("rl/bestnumpoly")

        
bestNumPoly(0)
        

In [ ]:
#Busca cual es el mejor número de ejemplos de entrenamiento, 
#dibujando gráficos.
def bestNumTraining():
    numTraining = np.array([5,10, 20,50,100,300, 
                            600, 1000, 1500, 2000, 2500,
                            3000, 3500])
    percentages = np.array([])
    percentagesT = np.array([])
    
    
    for i in numTraining:
        bestValues = chooseLanda(i, 0, plot=False)
        percentages = np.append(percentages, bestValues[0])
        percentagesT = np.append(percentagesT, bestValues[1])
    
    plt.figure()
    plt.xlabel("Number of training examples")
    plt.ylabel("Correct Percentage")
    plt.plot(numTraining, percentages, "-", c="orchid", label="Validation")
    plt.plot(numTraining, percentagesT, "-", c="darkseagreen", label="Training")
    plt.legend()
    plt.savefig("rl/NTrainingExamplesVSCorrectPercentage")


In [ ]:
bestNumTraining()

In [ ]:
chooseLanda(3000, 2)

# Comparaciones entre los 3 métodos

In [ ]:
def bestResults(rango, datos):
    #Regresión lineal
    bestPerRL = chooseLanda(3000, rango, data=datos, plot=False)[0]
    
    #Neural Network
    bestPerNN = neuralNetworkDecide(1000, 15, rango, data=datos, plot=False)
    
    #SVM
    bestPerSVM = SVM2(rango, data=datos)
        
    resultsArray = [bestPerRL, bestPerNN, bestPerSVM]

    plt.figure()
    plt.title("Best result obtained")
    plt.ylabel("Correct percentage (range=%i)" % rango)
    plt.ylim((np.min(resultsArray)-0.5, np.max(resultsArray)+0.5))
    plt.bar(["Regresión lineal", "Red neuronal", "Support Vector Machine"],
            [bestPerRL, bestPerNN, bestPerSVM], 
            color=["darkorchid", "gold", "dodgerblue"])
    plt.savefig("graph/comparisonrange%i" % rango)


In [ ]:
#Prueba con tres márgenes de errores y los mismos datos
#de entrenamiento
def bestResultsRangos():
    datos = loadDataShuffled()
    bestResults(0, datos)
    bestResults(1, datos)
    bestResults(2, datos)

In [ ]:
bestResultsRangos()